In [1]:
import pandas as pd
import numpy as np

In [2]:
webdata = pd.read_csv('../merged_data/cleaned_webdata.csv', sep=',') 
demodata = pd.read_csv('../merged_data/cleaned_data_demo.csv', sep=',')
clientsdata = pd.read_csv('../files_for_project/df_final_experiment_clients.csv', sep=',')

In [3]:
webdata.head()

,client_id,visitor_id,visit_id,process_step,date_time
0,9988021,580560515_7732621733,781255054_21935453173_531117,step_2,2017-04-17 15:26:51
1,9988021,580560515_7732621733,781255054_21935453173_531117,step_3,2017-04-17 15:19:22
2,9988021,580560515_7732621733,781255054_21935453173_531117,step_2,2017-04-17 15:19:13
3,9988021,580560515_7732621733,781255054_21935453173_531117,step_3,2017-04-17 15:18:04
4,9988021,580560515_7732621733,781255054_21935453173_531117,step_2,2017-04-17 15:17:15


In [4]:
demodata.head()

,client_id,clnt_tenure_yr,clnt_tenure_mnth,clnt_age,gendr,num_accts,bal,calls_6_mnth,logons_6_mnth
0,836976,6.0,73.0,60.5,NaN,2.0,45105.30,6.0,9.0
1,2304905,7.0,94.0,58.0,NaN,2.0,110860.30,6.0,9.0
2,1439522,5.0,64.0,32.0,NaN,2.0,52467.79,6.0,9.0
3,1562045,16.0,198.0,49.0,M,2.0,67454.65,3.0,6.0
4,5126305,12.0,145.0,33.0,F,2.0,103671.75,0.0,3.0


In [5]:

# Create a boolean mask to check which client_ids in webdata are present in demodata
webdata['is_client'] = webdata['client_id'].isin(demodata['client_id'])

# Optional: see how many are clients
num_clients = webdata['is_client'].sum()
print(f"Clients found in webdata: {num_clients} / {len(webdata)}")

Clients found in webdata: 443896 / 744640


In [6]:
webdata.head()

,client_id,visitor_id,visit_id,process_step,date_time,is_client
0,9988021,580560515_7732621733,781255054_21935453173_531117,step_2,2017-04-17 15:26:51,True
1,9988021,580560515_7732621733,781255054_21935453173_531117,step_3,2017-04-17 15:19:22,True
2,9988021,580560515_7732621733,781255054_21935453173_531117,step_2,2017-04-17 15:19:13,True
3,9988021,580560515_7732621733,781255054_21935453173_531117,step_3,2017-04-17 15:18:04,True
4,9988021,580560515_7732621733,781255054_21935453173_531117,step_2,2017-04-17 15:17:15,True


In [7]:
# Step 1: Get only rows in webdata that are from known clients
web_clients = webdata[webdata['is_client']]

# Step 2: Get unique client_ids (i.e. one entry per client)
unique_web_clients = web_clients[['client_id']].drop_duplicates()

# Step 3: Merge with demodata to get their demographic information
unique_client_profiles = unique_web_clients.merge(demodata, on='client_id', how='left')

# Step 4: Now you can calculate accurate stats
print("Unique clients using the online process:", len(unique_client_profiles))
print("Average age:", unique_client_profiles['clnt_age'].mean())
print("Gender distribution:\n", unique_client_profiles['gendr'].value_counts())
print("Average tenure (years):", unique_client_profiles['clnt_tenure_yr'].mean())

Unique clients using the online process: 70609
Average age: 46.442240133722414
Gender distribution:
 gendr
M    23724
F    22746
Name: count, dtype: int64
Average tenure (years): 12.052949925632127


In [8]:
# Group by exact age
age_counts = unique_client_profiles['clnt_age'].value_counts().sort_index()
                         
# OR group by age ranges
bins = [0, 18, 25, 35, 45, 60, 100]
labels = ['<18', '18-24', '25-34', '35-44', '45-59', '60+']
unique_client_profiles['age_group'] = pd.cut(unique_client_profiles['clnt_age'], bins=bins, labels=labels, right=False)

# Count visitors in each age group
age_group_counts = unique_client_profiles['age_group'].value_counts().sort_index()
print(age_group_counts)

age_group
<18        374
18-24     5545
25-34    14585
35-44    11874
45-59    21997
60+      16219
Name: count, dtype: int64


unique_client_profiles.head()

In [9]:
unique_client_profiles.head()

,client_id,clnt_tenure_yr,clnt_tenure_mnth,clnt_age,gendr,num_accts,bal,calls_6_mnth,logons_6_mnth,age_group
0,9988021,5.0,64.0,79.0,NaN,2.0,189023.86,1.0,4.0,60+
1,8320017,22.0,274.0,34.5,M,2.0,36001.90,5.0,8.0,25-34
2,4033851,12.0,149.0,63.5,M,2.0,142642.26,5.0,8.0,60+
3,1982004,6.0,80.0,44.5,NaN,2.0,30231.76,1.0,4.0,35-44
4,9294070,5.0,70.0,29.0,NaN,2.0,34254.54,0.0,3.0,25-34


In [10]:
# Merge the dataframes on the 'client_id' column with an inner join
full_info_dataframe_0 = pd.merge(webdata, demodata, on='client_id', how='inner')

In [11]:
full_info_dataframe = pd.merge(full_info_dataframe_0, clientsdata, on='client_id', how='inner')


In [12]:
full_info_dataframe.head()

,client_id,visitor_id,visit_id,process_step,date_time,is_client,clnt_tenure_yr,clnt_tenure_mnth,clnt_age,gendr,num_accts,bal,calls_6_mnth,logons_6_mnth,variation
0,9988021,580560515_7732621733,781255054_21935453173_531117,step_2,2017-04-17 15:26:51,True,5.0,64.0,79.0,NaN,2.0,189023.86,1.0,4.0,Test
1,9988021,580560515_7732621733,781255054_21935453173_531117,step_3,2017-04-17 15:19:22,True,5.0,64.0,79.0,NaN,2.0,189023.86,1.0,4.0,Test
2,9988021,580560515_7732621733,781255054_21935453173_531117,step_2,2017-04-17 15:19:13,True,5.0,64.0,79.0,NaN,2.0,189023.86,1.0,4.0,Test
3,9988021,580560515_7732621733,781255054_21935453173_531117,step_3,2017-04-17 15:18:04,True,5.0,64.0,79.0,NaN,2.0,189023.86,1.0,4.0,Test
4,9988021,580560515_7732621733,781255054_21935453173_531117,step_2,2017-04-17 15:17:15,True,5.0,64.0,79.0,NaN,2.0,189023.86,1.0,4.0,Test


In [13]:
full_info_dataframe.to_csv('../merged_data/full_info_dataframe.csv', index=False)
print("Data exported successfully!")

Data exported successfully!
